In [1]:
import matplotlib.pyplot as plt
from pprint import pprint as pp
import numpy as np
import pandas as pd

feature_matrix = []
target_vector1 = []
target_vector2 = []
varToNumNA = dict()

for line in open('CommViolPredUnnormalizedData.txt', 'r'):
    features_orig = line.strip().split(',')
    for i in range(len(features_orig)):
        if features_orig[i] == '?':
            try:
                varToNumNA[i] += 1
            except:
                varToNumNA[i] = 1
    
    target1 = features_orig[-2] # ViolentCrimesPerPop
    target2 = features_orig[-1] # nonViolPerPop
    #features = [ f for f in features[3:-2]] # don't include town and state name
    features = [ f for f in features_orig[4:103] ] #don't include 103
    features += [f for f in features_orig[120:123]] #don't include 103-119, or 123
    features += [features_orig[127]] 
    features += [f for f in features_orig[129:131]]
    features += [f for f in features_orig[133:143]]
    feature_matrix.append(features)
    target_vector1.append(target1)
    target_vector2.append(target2)
  

In [45]:
from sklearn.metrics import *
import numpy as np

# http://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
def statistical_measures(confusion_matrix):
    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.sum() - (FP + FN + TP)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)

    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return {'TPR':TPR, 'TNR':TNR, 'PPV':PPV, 'NPV':NPV, 'FPR':FPR, 'FNR':FNR, 'FDR':FDR, 'ACC':ACC}

In [2]:
# don't use the variables that have a lot of '?'s in th data
varToNumNA # {var : numNA}, var is the index of the variable, numNA is the nubmer of ?s

{2: 1221,
 3: 1224,
 30: 1,
 103: 1872,
 104: 1872,
 105: 1872,
 106: 1872,
 107: 1872,
 108: 1872,
 109: 1872,
 110: 1872,
 111: 1872,
 112: 1872,
 113: 1872,
 114: 1872,
 115: 1872,
 116: 1872,
 117: 1872,
 118: 1872,
 119: 1872,
 123: 1872,
 124: 1872,
 125: 1872,
 126: 1872,
 128: 1872,
 131: 208,
 132: 208,
 133: 1,
 134: 1,
 135: 13,
 136: 13,
 137: 3,
 138: 3,
 139: 3,
 140: 3,
 141: 3,
 142: 3,
 143: 91,
 144: 91,
 145: 221,
 146: 97}

In [3]:
len(features)

115

In [4]:
feature_matrix[1]
'?' in feature_matrix[1]

False

In [5]:
feature_matrix_clean = []
target_vector1_clean = []
target_vector2_clean = []
for i in range(len(feature_matrix)):
    if ('?' not in feature_matrix[i] and '?' not in target_vector1[i] and '?' not in target_vector2[i]):
        feature_matrix_clean.append([float(x) for x in feature_matrix[i]])
        target_vector1_clean.append(float(target_vector1[i]))
        target_vector2_clean.append(float(target_vector2[i]))

In [6]:
len(feature_matrix), len(feature_matrix_clean) # get rid of some data ~300

(2215, 1901)

In [8]:
AVG_CRIME = 636.6

In [9]:
data = np.array( feature_matrix_clean )
target1 = np.array( [ (1 if (x > AVG_CRIME) else 0) for x in target_vector1_clean] )
target2 = np.array( [ (1 if (x > AVG_CRIME) else 0) for x in target_vector2_clean] )

print(data)
print(target1)

[[  1.00000000e+00   1.19800000e+04   3.10000000e+00 ...,   1.13208000e+03
    1.60000000e+01   1.31260000e+02]
 [  1.00000000e+00   2.31230000e+04   2.82000000e+00 ...,   1.59878000e+03
    2.60000000e+01   1.10550000e+02]
 [  1.00000000e+00   2.93440000e+04   2.43000000e+00 ...,   4.97219000e+03
    1.36000000e+02   3.76300000e+02]
 ..., 
 [  1.00000000e+01   3.28240000e+04   2.46000000e+00 ...,   2.43597000e+03
    1.79000000e+02   4.87190000e+02]
 [  1.00000000e+01   1.35470000e+04   2.89000000e+00 ...,   3.72290000e+03
    1.30000000e+01   1.02100000e+02]
 [  1.00000000e+01   2.88980000e+04   2.61000000e+00 ...,   4.81920000e+03
    4.05000000e+02   1.33867000e+03]]
[0 0 0 ..., 0 1 1]


In [10]:
# We will use a variation of NB 
from sklearn.naive_bayes import GaussianNB

# fit a Naive Bayes model to the data
model = GaussianNB()
X_train, y_train1 = data, target1 
model.fit(X_train, y_train1)

GaussianNB(priors=None)

In [11]:
y_predicted = model.predict(X_train) 
y_expected = y_train1

In [12]:
# Import  metrics
from sklearn import metrics

# summarize the fit of the model

print(metrics.accuracy_score(y_expected, y_predicted))
print()
print(metrics.classification_report(y_expected, y_predicted))
print(metrics.confusion_matrix(y_expected, y_predicted))
print()

0.867964229353

             precision    recall  f1-score   support

          0       0.86      0.97      0.91      1305
          1       0.90      0.65      0.76       596

avg / total       0.87      0.87      0.86      1901

[[1263   42]
 [ 209  387]]



In [13]:
def readFile(filename, mode="rt"):
    # rt stands for "read text"
    fin = contents = None
    try:
        fin = open(filename, mode)
        contents = fin.read()
    finally:
        if (fin != None): fin.close()
    return contents

#def indexToName(i):
#    contents = readFile('varNames.txt')
#    contents_list = contents.split('\n')
#    contents_list = [ (s.split())[1][:-1] for s in contents_list ]
#    return contents_list[i]

# get all of the variable names
contents = readFile('varNames.txt')
contents_list = contents.split('\n')
contents_list = [ (s.split())[1][:-1] for s in contents_list ]
#contents_list.index('population')

In [14]:
varNames = []
for i in varToNumNA:
    varNames += [(i, varToNumNA[i], contents_list[i])]
sorted(varNames) # variables that we didn't use: (index, # of times used, var name)

[(2, 1221, 'countyCode'),
 (3, 1224, 'communityCode'),
 (30, 1, 'OtherPerCap'),
 (103, 1872, 'LemasSwornFT'),
 (104, 1872, 'LemasSwFTPerPop'),
 (105, 1872, 'LemasSwFTFieldOps'),
 (106, 1872, 'LemasSwFTFieldPerPop'),
 (107, 1872, 'LemasTotalReq'),
 (108, 1872, 'LemasTotReqPerPop'),
 (109, 1872, 'PolicReqPerOffic'),
 (110, 1872, 'PolicPerPop'),
 (111, 1872, 'RacialMatchCommPol'),
 (112, 1872, 'PctPolicWhite'),
 (113, 1872, 'PctPolicBlack'),
 (114, 1872, 'PctPolicHisp'),
 (115, 1872, 'PctPolicAsian'),
 (116, 1872, 'PctPolicMinor'),
 (117, 1872, 'OfficAssgnDrugUnits'),
 (118, 1872, 'NumKindsDrugsSeiz'),
 (119, 1872, 'PolicAveOTWorked'),
 (123, 1872, 'PolicCars'),
 (124, 1872, 'PolicOperBudg'),
 (125, 1872, 'LemasPctPolicOnPatr'),
 (126, 1872, 'LemasGangUnitDeploy'),
 (128, 1872, 'PolicBudgPerPop'),
 (131, 208, 'rapes'),
 (132, 208, 'rapesPerPop'),
 (133, 1, 'robberies'),
 (134, 1, 'robbbPerPop'),
 (135, 13, 'assaults'),
 (136, 13, 'assaultPerPop'),
 (137, 3, 'burglaries'),
 (138, 3, 'burgl

In [46]:
from sklearn import linear_model, datasets
# use logistic reg and L1 penalty 
logreg = linear_model.LogisticRegression(C=1e5, penalty='l1',)
X = feature_matrix_clean
y = [ (1 if (x > AVG_CRIME) else 0) for x in target_vector1_clean]
logreg.fit(X, y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [47]:
y_predicted_log = logreg.predict(X)
y_expected = y_train1
print(metrics.accuracy_score(y_expected, y_predicted_log))
print()
print(metrics.classification_report(y_expected, y_predicted_log))
print(metrics.confusion_matrix(y_expected, y_predicted_log))
print()

0.992635455024

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1305
          1       0.99      0.98      0.99       596

avg / total       0.99      0.99      0.99      1901

[[1301    4]
 [  10  586]]



In [48]:
# logistic regression w/ L1 penalty and CV
from sklearn import cross_validation
predicted = cross_validation.cross_val_predict(linear_model.LogisticRegression(penalty='l1'), X, y, cv=10)
print(metrics.accuracy_score(y, predicted))
print(metrics.classification_report(y, predicted))

0.97475013151
             precision    recall  f1-score   support

          0       0.98      0.98      0.98      1305
          1       0.97      0.95      0.96       596

avg / total       0.97      0.97      0.97      1901



In [49]:
# l1 log reg, CV
cm2 = confusion_matrix(y, predicted)
statistical_measures(cm2)

{'ACC': array([ 0.97475013,  0.97475013]),
 'FDR': array([ 0.02132521,  0.03401361]),
 'FNR': array([ 0.01532567,  0.04697987]),
 'FPR': array([ 0.04697987,  0.01532567]),
 'NPV': array([ 0.96598639,  0.97867479]),
 'PPV': array([ 0.97867479,  0.96598639]),
 'TNR': array([ 0.95302013,  0.98467433]),
 'TPR': array([ 0.98467433,  0.95302013])}

In [50]:
# use logistic reg and L2 penalty 
logreg2 = linear_model.LogisticRegression(C=1e5, penalty='l2',)
logreg2.fit(X, y)

y_predicted_log2 = logreg2.predict(X)
print(metrics.accuracy_score(y_expected, y_predicted_log2))
print()
print(metrics.classification_report(y_expected, y_predicted_log2))
print(metrics.confusion_matrix(y_expected, y_predicted_log2))
print()

0.990005260389

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1305
          1       0.99      0.98      0.98       596

avg / total       0.99      0.99      0.99      1901

[[1299    6]
 [  13  583]]



In [51]:
predicted2 = cross_validation.cross_val_predict(linear_model.LogisticRegression(penalty='l2'), X, y, cv=10)
print(metrics.accuracy_score(y, predicted2))
print(metrics.classification_report(y, predicted2))

0.978432403998
             precision    recall  f1-score   support

          0       0.98      0.99      0.98      1305
          1       0.97      0.96      0.97       596

avg / total       0.98      0.98      0.98      1901



In [52]:
# l2 log reg, CV
cm3 = confusion_matrix(y, predicted2)
statistical_measures(cm3)

{'ACC': array([ 0.9784324,  0.9784324]),
 'FDR': array([ 0.01681957,  0.03204047]),
 'FNR': array([ 0.01455939,  0.03691275]),
 'FPR': array([ 0.03691275,  0.01455939]),
 'NPV': array([ 0.96795953,  0.98318043]),
 'PPV': array([ 0.98318043,  0.96795953]),
 'TNR': array([ 0.96308725,  0.98544061]),
 'TPR': array([ 0.98544061,  0.96308725])}

In [20]:
# now try what we did in class on 5/2 (random forest and confusion matrix to analyze)
df = pd.read_csv('CommViolPredUnnormalizedData.txt', header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.10,1.37,91.78,6.50,...,14,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
1,Marpletownship,PA,45,47616,1,23123,2.82,0.80,95.57,3.44,...,57,242.37,376,1598.78,26,110.55,1,4.25,127.56,1955.95
2,Tigardcity,OR,?,?,1,29344,2.43,0.74,94.33,3.43,...,274,758.14,1797,4972.19,136,376.3,22,60.87,218.59,6167.51
3,Gloversvillecity,NY,35,29443,1,16656,2.40,1.70,97.35,0.50,...,225,1301.78,716,4142.56,47,271.93,?,?,306.64,?
4,Bemidjicity,MN,7,5068,1,11245,2.76,0.53,89.16,1.17,...,91,728.93,1060,8490.87,91,728.93,5,40.05,?,9988.79


In [21]:
df.columns = contents_list # add headers with correct variable names
df.head(5)

,communityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.10,1.37,91.78,6.50,...,14,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
1,Marpletownship,PA,45,47616,1,23123,2.82,0.80,95.57,3.44,...,57,242.37,376,1598.78,26,110.55,1,4.25,127.56,1955.95
2,Tigardcity,OR,?,?,1,29344,2.43,0.74,94.33,3.43,...,274,758.14,1797,4972.19,136,376.3,22,60.87,218.59,6167.51
3,Gloversvillecity,NY,35,29443,1,16656,2.40,1.70,97.35,0.50,...,225,1301.78,716,4142.56,47,271.93,?,?,306.64,?
4,Bemidjicity,MN,7,5068,1,11245,2.76,0.53,89.16,1.17,...,91,728.93,1060,8490.87,91,728.93,5,40.05,?,9988.79


In [22]:
onlyVarNames = [ v[2] for v in varNames ] # get the variables that we don't use bc they have too many NAs

In [23]:
df2 = df.drop(onlyVarNames+['communityname', 'state'], axis=1) # drop vars that have a lot of NAs
df.shape, df2.shape

((2215, 147), (2215, 104))

In [24]:
df2.head(10)

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,murders,murdPerPop
0,1,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,...,53.72,65.29,78.09,89.14,6.5,1845.9,9.63,0.00,0,0.00
1,1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,...,77.17,71.27,90.22,96.12,10.6,2186.7,3.84,0.00,0,0.00
2,1,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,...,44.77,36.60,61.26,82.85,10.6,2780.9,4.37,0.00,3,8.30
3,1,16656,2.40,1.70,97.35,0.50,0.70,12.55,25.20,12.19,...,88.71,56.70,90.17,96.24,5.2,3217.7,3.31,0.00,0,0.00
4,1,11245,2.76,0.53,89.16,1.17,0.52,24.46,40.53,28.69,...,73.75,42.22,60.34,89.02,11.5,974.2,0.38,0.00,0,0.00
5,1,140494,2.45,2.51,95.65,0.90,0.95,18.09,32.89,20.04,...,64.35,42.29,70.61,85.66,70.4,1995.7,0.97,0.00,7,4.63
6,1,28700,2.60,1.60,96.57,1.47,1.10,11.17,27.41,12.76,...,77.30,63.45,82.23,93.53,10.9,2643.5,9.62,0.00,0,0.00
7,1,59459,2.45,14.20,84.87,0.40,0.63,15.31,27.93,14.78,...,73.70,54.85,85.55,91.51,39.2,1515.3,0.70,0.00,8,13.13
8,1,74111,2.46,0.35,97.11,1.25,0.73,16.64,35.16,20.33,...,58.82,40.72,67.97,81.39,30.9,2399.3,1.41,0.00,0,0.00
9,1,103590,2.62,23.14,67.60,0.92,16.35,19.88,34.55,21.62,...,75.59,42.33,74.05,92.12,78.5,1319.3,0.76,6.57,29,26.88


In [25]:
# double check that there are no '?'s (NAs)
df2 = df2.replace('?', np.nan)
df2 = df2.dropna(axis=0)
df2.shape

(2215, 104)

In [26]:
X = df2[df.ViolentCrimesPerPop != '?'] # didn't get rid of '?' in the y (ViolentCrimesPerPop) yet
y = df.ViolentCrimesPerPop[df.ViolentCrimesPerPop != '?']
y = pd.Series([float(a) > AVG_CRIME for a in y ]) # make the y 0 or 1
X.shape, y.shape

((1994, 104), (1994,))

In [27]:
# X.dtypes # check that datatypes are numbers

In [28]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=364)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1595, 104), (399, 104), (1595,), (399,))

In [30]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [31]:
predicted_rf = clf.predict(X_test)

In [34]:
# random forest 
cm1 = confusion_matrix(y_test, predicted_rf)

In [35]:
statistical_measures(cm1)

{'ACC': array([ 0.87218045,  0.87218045]),
 'FDR': array([ 0.1137931 ,  0.16513761]),
 'FNR': array([ 0.06545455,  0.26612903]),
 'FPR': array([ 0.26612903,  0.06545455]),
 'NPV': array([ 0.83486239,  0.8862069 ]),
 'PPV': array([ 0.8862069 ,  0.83486239]),
 'TNR': array([ 0.73387097,  0.93454545]),
 'TPR': array([ 0.93454545,  0.73387097])}

In [36]:
# random forest w/ CV
predicted_rf_cv = cross_validation.cross_val_predict(RandomForestClassifier(n_estimators=20), X, y, cv=10)
print(metrics.accuracy_score(y, predicted_rf_cv))
print(metrics.classification_report(y, predicted_rf_cv))

0.861083249749
             precision    recall  f1-score   support

      False       0.87      0.93      0.90      1362
       True       0.83      0.71      0.76       632

avg / total       0.86      0.86      0.86      1994



In [40]:
cm1b = confusion_matrix(y, predicted_rf_cv)
statistical_measures(cm1b)

{'ACC': array([ 0.86108325,  0.86108325]),
 'FDR': array([ 0.12611992,  0.17311234]),
 'FNR': array([ 0.06901615,  0.28955696]),
 'FPR': array([ 0.28955696,  0.06901615]),
 'NPV': array([ 0.82688766,  0.87388008]),
 'PPV': array([ 0.87388008,  0.82688766]),
 'TNR': array([ 0.71044304,  0.93098385]),
 'TPR': array([ 0.93098385,  0.71044304])}

In [ ]:
def cm_analysis(confusion_matrix):
    FP = confusion_matrix[0][1]
    FN = confusion_matrix[1][0]
    TP = confusion_matrix[1][1]
    TN = confusion_matrix[0][0]

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)

    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return {'true positive':TPR, 'true negative':TNR, 'precision':PPV, 'negative predictive val':NPV, 'false positive':FPR, 'false negative':FNR, 'false discovery':FDR}

In [ ]:
cm_analysis(cm2)

In [ ]:
statistical_measures(cm2)